<a href="https://colab.research.google.com/github/Ry02024/Conv-with-AI/blob/main/conv_with_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 初期設定

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 環境構築

In [3]:
!pip install streamlit -q
!pip install pyngrok -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00


In [4]:
import streamlit as st
from pyngrok import ngrok

In [6]:
from google.colab import userdata
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')

In [7]:
from pyngrok import ngrok
import os

# ngrokトークンを設定
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# 実行ファイル

In [81]:
%%writefile streamlit_app.py
import streamlit as st
import google.generativeai as genai

# タイトルと説明
st.title("💬 シンプル対話システム")
st.write("A さんの意見を元に AI が考察を提供し、B さんが修正・承認する対話システム")

# Gemini APIキーの入力
gemini_api_key = st.text_input("Gemini API Key", type="password")

# セッションの初期化
if "messages" not in st.session_state:
    st.session_state.messages = []
if "waiting_for_ai" not in st.session_state:
    st.session_state.waiting_for_ai = False  # AI が考察中かどうか
if "waiting_for_b" not in st.session_state:
    st.session_state.waiting_for_b = False  # B さんの承認待ち状態
if "b_review_text" not in st.session_state:
    st.session_state.b_review_text = ""  # B さんの修正用のテキスト

# APIキーが入力された場合
if gemini_api_key:
    try:
        # Geminiクライアントの設定
        genai.configure(api_key=gemini_api_key)
        model = genai.GenerativeModel(model_name='gemini-1.5-pro')

        # チャット履歴の表示
        st.subheader("📜 Chat History")
        for message in st.session_state.messages:
            role = "A さん" if message["role"] == "user" else "AI" if message["role"] == "assistant" else "B さん"
            with st.chat_message(role):
                st.markdown(message["content"])

        # **A さんの意見入力（エンターで送信）**
        if not st.session_state.waiting_for_ai and not st.session_state.waiting_for_b:
            user_input = st.chat_input("A さんの意見を入力してください:")

            if user_input:  # 空入力や None の場合は処理しない
                st.session_state.messages.append({"role": "user", "content": user_input})
                st.session_state.waiting_for_ai = True  # AI の処理待ち状態に切り替え
                st.rerun()  # 画面を更新して AI の考察に進む

        # **A さんの発言後、AI が考察**
        elif st.session_state.waiting_for_ai:
            with st.spinner("AIが考えています..."):  # AI の考察中にスピナー表示
                # AIに過去の履歴を含めて文脈を伝える
                conversation_history = "\n".join(
                    [f"{m['role']}: {m['content']}" for m in st.session_state.messages]
                )
                prompt = f"""
                現在の会話の文脈:
                {conversation_history}

                ユーザーが「{st.session_state.messages[-1]['content']}」と発言しました。
                適切な文脈を考慮し、簡潔に回答してください。
                """

                # AIが考察を生成
                try:
                    response = model.generate_content(contents=[prompt])
                    ai_thoughts = response.text if response.text else "AIの考察を生成できませんでした。"

                    # AIの考察をセッションに保存し、B さんが編集できるようにする
                    st.session_state.messages.append({"role": "assistant", "content": ai_thoughts})
                    st.session_state.b_review_text = ai_thoughts  # AIの考察をB さんの入力として保持
                    st.session_state.waiting_for_ai = False  # AI の処理終了
                    st.session_state.waiting_for_b = True  # B さんの修正フェーズに移行
                    st.rerun()  # 画面を更新して B さんの入力フェーズへ

                except Exception as e:
                    st.error(f"エラーが発生しました: {e}")
                    st.session_state.waiting_for_ai = False  # エラー時にも状態リセット
                    st.rerun()

        # **B さんが修正 & 承認（エンターで確定）**
        elif st.session_state.waiting_for_b:
            st.write("📝 B さんの修正エリア（AIの考察が反映）:")
            b_review_text = st.text_area(
                "B さんが修正してください:",
                value=st.session_state.b_review_text,  # AI の考察を事前入力
                height=150
            )

            if st.chat_input("B さんが修正を確定:"):
                st.session_state.messages.append({"role": "b_user", "content": b_review_text})
                st.session_state.b_review_text = ""  # 修正完了後リセット
                st.session_state.waiting_for_b = False  # 次の A さんの入力を待つ状態に戻す
                st.rerun()  # 画面を更新して A さんの入力フェーズへ

    except Exception as e:
        st.error(f"Gemini API の設定に失敗しました: {e}")

else:
    st.info("Gemini APIキーを入力してください", icon="🗝️")


Overwriting streamlit_app.py


#アプリの起動

In [82]:
# ngrokを介してStreamlitを公開
public_url = ngrok.connect(addr='8501')
print('Public URL:', public_url)

Public URL: NgrokTunnel: "https://8623-34-80-101-136.ngrok-free.app" -> "http://localhost:8501"


In [83]:
# Streamlitアプリケーションの起動
!streamlit run streamlit_app.py >/dev/null

^C
